In [5]:
pip install basketball-reference-scraper

     ---------------------------------------- 51.9/51.9 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from IPython.display import display, Markdown
import matplotlib.colors as mcolors

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Calculate per-game statistics before dropping 'G'
    if 'G' in team_stats.columns:
        for col in per_game_columns:
            if col in team_stats.columns:  # Ensure the column exists
                team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop the 'G' and 'MP' columns after calculations
    team_stats.drop(columns=['G', 'MP'], inplace=True, errors='ignore')

    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Filter rows for Raptors 905 and League Average
    filtered_teams = team_stats.loc[team_stats['Team'].isin(['Raptors 905', 'League Average'])]
    
else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

filtered_teams

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
10,Raptors 905,43.500,91.625,0.475,12.750,36.875,0.346,30.750,54.750,0.562,8.375,13.0,0.644,27.375,12.25,7.25,16.125,113.250
31,League Average,42.375,93.625,0.453,13.375,39.500,0.338,29.125,54.125,0.537,11.250,15.0,0.749,24.625,9.75,6.00,10.875,116.625


In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # Specify columns where ranking should be ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Color coding logic
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for specified columns
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply the coloring logic to the DataFrame with ranks
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
0,Valley Suns,44.625 (2),91.375 (9),0.488 (2),18.75 (1),44.0 (4),0.426 (1),25.875 (23),47.375 (25),0.546 (13),10.375 (16),14.0 (16),0.741 (20),25.75 (6),8.25 (23),4.125 (26),17.375 (31),125.125 (1)
1,Westchester Knicks,46.5 (1),98.25 (1),0.473 (7),11.0 (24),34.25 (25),0.321 (21),35.5 (1),64.0 (1),0.555 (8),9.5 (22),14.375 (13),0.661 (29),25.75 (6),10.0 (11),8.0 (3),12.25 (23),120.125 (2)
2,San Diego Clippers,41.5 (8),91.5 (7),0.454 (13),12.625 (15),36.125 (18),0.349 (11),28.875 (10),55.375 (6),0.521 (24),14.125 (2),20.375 (1),0.693 (27),22.875 (17),12.625 (3),3.75 (29),4.25 (2),118.625 (3)
3,Rip City Remix,42.75 (6),94.75 (2),0.451 (15),14.25 (7),45.0 (2),0.317 (25),28.5 (11),49.75 (21),0.573 (3),10.625 (14),13.625 (18),0.78 (8),20.875 (26),8.5 (19),6.25 (11),14.5 (28),117.0 (4)
4,Sioux Falls Skyforce,44.0 (3),87.444 (20),0.503 (1),12.0 (19),34.222 (26),0.351 (8),32.0 (3),53.222 (12),0.601 (1),10.0 (19),12.889 (23),0.776 (9),29.333 (1),10.333 (7),5.222 (19),9.889 (14),116.778 (5)
5,Motor City Cruise,43.111 (5),89.889 (12),0.48 (4),13.222 (10),35.556 (20),0.372 (3),29.889 (8),54.333 (10),0.55 (11),9.444 (24),12.0 (26),0.787 (6),25.667 (8),10.111 (9),4.889 (21),5.222 (3),115.556 (6)
6,Stockton Kings,41.375 (9),87.5 (19),0.473 (7),15.625 (4),38.125 (11),0.41 (2),25.75 (24),49.375 (23),0.522 (22),10.125 (18),13.5 (20),0.75 (17),26.25 (4),7.125 (29),6.0 (13),12.875 (24),115.125 (7)
7,Salt Lake City Stars,41.375 (9),91.5 (7),0.452 (14),11.25 (21),35.0 (21),0.321 (21),30.125 (7),56.5 (5),0.533 (17),12.25 (5),15.25 (8),0.803 (5),23.0 (15),10.375 (6),3.75 (29),9.625 (13),114.75 (8)
8,Iowa Wolves,42.2 (7),87.6 (18),0.482 (3),9.6 (29),28.5 (30),0.337 (14),32.6 (2),59.1 (3),0.552 (9),11.9 (8),16.1 (5),0.739 (22),25.1 (9),10.1 (10),5.9 (14),13.6 (25),114.4 (9)
9,Long Island Nets,41.333 (11),92.667 (5),0.446 (18),10.889 (25),34.778 (22),0.313 (26),30.444 (5),57.889 (4),0.526 (21),11.889 (9),15.222 (9),0.781 (7),24.556 (10),8.444 (20),5.333 (18),14.667 (29),113.778 (10)


In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Filter the row for Raptors 905 only
    raptors_row = team_stats.loc[team_stats['Team'] == 'Raptors 905']
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # Specify columns where ranking should be ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Apply color coding logic to Raptors 905 row
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for specified columns
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Style the Raptors 905 row
    styled_df = df_with_ranks.loc[df_with_ranks['Team'] == 'Raptors 905'].style.apply(
        color_cells, subset=numeric_cols, axis=1
    ).format(precision=3)

else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
10,Raptors 905,43.5 (4),91.625 (6),0.475 (5),12.75 (14),36.875 (16),0.346 (13),30.75 (4),54.75 (7),0.562 (4),8.375 (29),13.0 (22),0.644 (31),27.375 (2),12.25 (4),7.25 (6),16.125 (31),113.25 (11)


In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Filter the row for opponent 905 only
    opponent_row = team_stats.loc[team_stats['Team'] == 'Greensboro Swarm']
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # Specify columns where ranking should be ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Apply color coding logic to opponent 905 row
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for specified columns
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Style the opponent 905 row
    styled_df = df_with_ranks.loc[df_with_ranks['Team'] == 'Greensboro Swarm'].style.apply(
        color_cells, subset=numeric_cols, axis=1
    ).format(precision=3)

else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
18,Greensboro Swarm,39.25 (18),87.125 (21),0.451 (15),9.0 (30),27.125 (31),0.332 (17),30.25 (6),60.0 (2),0.504 (28),14.125 (2),18.75 (2),0.753 (15),15.25 (31),10.125 (7),4.0 (27),3.625 (1),109.125 (19)


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'G', 'MP', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)

else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

team_stats

,Team,G,MP,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
0,Valley Suns,8,1945,44.625,91.375,0.488,18.750,44.000,0.426,25.875,47.375,0.546,10.375,14.000,0.741,25.750,8.250,4.125,17.375,125.125
1,Westchester Knicks,8,1920,46.500,98.250,0.473,11.000,34.250,0.321,35.500,64.000,0.555,9.500,14.375,0.661,25.750,10.000,8.000,12.250,120.125
2,San Diego Clippers,8,1945,41.500,91.500,0.454,12.625,36.125,0.349,28.875,55.375,0.521,14.125,20.375,0.693,22.875,12.625,3.750,4.250,118.625
3,Rip City Remix,8,1920,42.750,94.750,0.451,14.250,45.000,0.317,28.500,49.750,0.573,10.625,13.625,0.780,20.875,8.500,6.250,14.500,117.000
4,Sioux Falls Skyforce,9,2185,44.000,87.444,0.503,12.000,34.222,0.351,32.000,53.222,0.601,10.000,12.889,0.776,29.333,10.333,5.222,9.889,116.778
5,Motor City Cruise,9,2210,43.111,89.889,0.480,13.222,35.556,0.372,29.889,54.333,0.550,9.444,12.000,0.787,25.667,10.111,4.889,5.222,115.556
6,Stockton Kings,8,1920,41.375,87.500,0.473,15.625,38.125,0.410,25.750,49.375,0.522,10.125,13.500,0.750,26.250,7.125,6.000,12.875,115.125
7,Salt Lake City Stars,8,1920,41.375,91.500,0.452,11.250,35.000,0.321,30.125,56.500,0.533,12.250,15.250,0.803,23.000,10.375,3.750,9.625,114.750
8,Iowa Wolves,10,2425,42.200,87.600,0.482,9.600,28.500,0.337,32.600,59.100,0.552,11.900,16.100,0.739,25.100,10.100,5.900,13.600,114.400
9,Long Island Nets,9,2160,41.333,92.667,0.446,10.889,34.778,0.313,30.444,57.889,0.526,11.889,15.222,0.781,24.556,8.444,5.333,14.667,113.778


# Standard Stats Against

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-opponent'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Calculate per-game statistics before dropping 'G'
    if 'G' in team_stats.columns:
        for col in per_game_columns:
            if col in team_stats.columns:  # Ensure the column exists
                team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'FTM/g', 'FT%']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop the 'G' and 'MP' columns after calculations
    team_stats.drop(columns=['G', 'MP'], inplace=True, errors='ignore')

    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTA/g', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Filter rows for Raptors 905 and League Average
    filtered_teams = team_stats.loc[team_stats['Team'].isin(['Raptors 905', 'League Average'])]
    
else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

filtered_teams

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTA/g,AST/g,STL/g,BLK/g,PF/g,PTS/g
28,Raptors 905,41.875,89.375,0.469,13.875,36.250,0.383,28.0,53.125,0.527,18.125,25.000,8.0,5.875,7.375,119.000
31,League Average,39.444,87.000,0.453,12.444,36.889,0.339,27.0,50.222,0.537,13.889,22.889,9.0,5.556,10.111,108.444


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-opponent'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop 'FTM/g' and 'FT%' columns
    team_stats.drop(columns=['FTM/g', 'FT%'], inplace=True, errors='ignore')

    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTA/g', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # PF/g will be descending, all others ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Color coding logic
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for PF/g (descending order)
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply the coloring logic to the DataFrame with ranks
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

else:
    print("Table with id 'team-stats-sc-opponent' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTA/g,AST/g,STL/g,BLK/g,PF/g,PTS/g
0,College Park Skyhawks,36.667 (3),87.111 (9),0.421 (2),9.333 (1),34.0 (6),0.275 (1),27.333 (15),53.111 (19),0.515 (6),15.111 (21),20.889 (4),9.333 (19),4.444 (6),13.444 (7),100.667 (1)
1,Greensboro Swarm,35.625 (1),81.375 (2),0.438 (10),11.375 (9),36.125 (13),0.315 (8),24.25 (4),45.25 (5),0.536 (17),15.125 (22),18.375 (1),7.75 (4),6.0 (18),20.5 (1),101.375 (2)
2,Birmingham Squadron,36.889 (4),87.889 (13),0.42 (1),14.667 (25),40.889 (23),0.359 (24),22.222 (2),47.0 (9),0.473 (1),10.778 (1),22.111 (9),10.0 (22),2.556 (1),8.556 (21),103.333 (3)
3,Iowa Wolves,38.0 (5),86.6 (8),0.439 (11),10.3 (4),34.0 (6),0.303 (4),27.7 (17),52.6 (16),0.527 (10),14.5 (17),24.4 (22),10.1 (24),6.2 (23),10.0 (14),104.5 (4)
4,Wisconsin Herd,38.125 (6),87.5 (11),0.436 (8),10.0 (2),31.375 (3),0.319 (10),28.125 (19),56.125 (26),0.501 (3),15.875 (27),22.5 (11),8.25 (9),6.75 (26),11.75 (10),104.625 (5)
5,Memphis Hustle,36.2 (2),83.4 (4),0.434 (7),15.5 (30),43.5 (28),0.356 (23),20.7 (1),39.9 (1),0.519 (9),13.3 (11),23.8 (17),7.9 (5),7.3 (28),15.2 (3),105.4 (6)
6,Austin Spurs,39.111 (11),81.111 (1),0.482 (29),13.556 (19),34.444 (8),0.394 (29),25.556 (8),46.667 (7),0.548 (23),11.556 (3),19.222 (2),8.0 (6),5.889 (17),11.111 (11),106.111 (7)
7,Sioux Falls Skyforce,39.444 (12),87.111 (9),0.453 (15),10.0 (2),32.556 (5),0.307 (5),29.444 (25),54.556 (24),0.54 (19),13.556 (14),22.778 (14),8.889 (17),5.333 (11),9.556 (15),106.222 (8)
8,Mexico City Capitanes,38.444 (8),89.0 (16),0.432 (6),12.444 (15),42.889 (25),0.29 (2),26.0 (11),46.111 (6),0.564 (25),13.778 (15),21.444 (6),12.0 (30),5.556 (12),7.333 (25),106.667 (9)
9,Oklahoma City Blue,39.444 (12),90.556 (24),0.436 (8),14.0 (22),45.444 (31),0.308 (6),25.444 (7),45.111 (4),0.564 (25),12.0 (5),22.444 (10),10.0 (22),6.778 (27),7.111 (27),107.0 (10)


In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-opponent'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop 'FTM/g' and 'FT%' columns
    team_stats.drop(columns=['FTM/g', 'FT%'], inplace=True, errors='ignore')

    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTA/g', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # PF/g will be descending, all others ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Apply color coding logic to Raptors 905 row
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for PF/g (descending order)
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Filter for Raptors 905 only
    raptors_row = df_with_ranks[df_with_ranks['Team'] == 'Raptors 905']
    
    # Style the Raptors 905 row
    styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

else:
    print("Table with id 'team-stats-sc-opponent' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTA/g,AST/g,STL/g,BLK/g,PF/g,PTS/g
28,Raptors 905,41.875 (24),89.375 (18),0.469 (22),13.875 (20),36.25 (14),0.383 (28),28.0 (18),53.125 (20),0.527 (10),18.125 (31),25.0 (25),8.0 (6),5.875 (16),7.375 (24),119.0 (29)


In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-opponent'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop 'FTM/g' and 'FT%' columns
    team_stats.drop(columns=['FTM/g', 'FT%'], inplace=True, errors='ignore')

    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTA/g', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # PF/g will be descending, all others ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Apply color coding logic to opponent 905 row
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for PF/g (descending order)
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Filter for opponent 905 only
    opponent_row = df_with_ranks[df_with_ranks['Team'] == 'Greensboro Swarm']
    
    # Style the opponent 905 row
    styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

else:
    print("Table with id 'team-stats-sc-opponent' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTA/g,AST/g,STL/g,BLK/g,PF/g,PTS/g
1,Greensboro Swarm,35.625 (1),81.375 (2),0.438 (10),11.375 (9),36.125 (13),0.315 (8),24.25 (4),45.25 (5),0.536 (17),15.125 (22),18.375 (1),7.75 (4),6.0 (18),20.5 (1),101.375 (2)


In [10]:
team_stats.columns

Index(['Rk', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'Pace', 'ORtg'],
      dtype='object')

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    print(team_stats)  # Display the DataFrame
else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()


      Rk                      Team   G    MP   FG  FGA    FG%   3P  3PA  \
0    1.0               Valley Suns   8  1945  357  731  0.488  150  352   
1    2.0        Westchester Knicks   8  1920  372  786  0.473   88  274   
2    3.0        San Diego Clippers   8  1945  332  732  0.454  101  289   
3    4.0            Rip City Remix   8  1920  342  758  0.451  114  360   
4    5.0      Sioux Falls Skyforce   9  2185  396  787  0.503  108  308   
5    6.0         Motor City Cruise   9  2210  388  809  0.480  119  320   
6    7.0            Stockton Kings   8  1920  331  700  0.473  125  305   
7    8.0      Salt Lake City Stars   8  1920  331  732  0.452   90  280   
8    9.0               Iowa Wolves  10  2425  422  876  0.482   96  285   
9   10.0          Long Island Nets   9  2160  372  834  0.446   98  313   
10  11.0               Raptors 905   8  1920  348  733  0.475  102  295   
11  12.0              Austin Spurs   9  2185  370  841  0.440  117  390   
12  13.0             Main

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,Iowa Wolves,114.4 (8),42.2 (7),88.1 (17),47.9 (2),9.6 (29),28.5 (30),33.7 (14),12.0 (7),16.2 (4),74.1 (20),12.5 (16),34.9 (6),47.4 (8),24.7 (11),17.3 (15),10.0 (6),6.3 (12),6.6 (9),22.2 (27),22.8 (4),9.9 (2)
1,Greensboro Swarm,109.1 (19),39.3 (18),87.6 (22),44.8 (15),9.0 (30),27.0 (31),33.3 (15),14.1 (2),18.8 (2),75.3 (15),14.4 (7),33.9 (12),48.3 (5),17.0 (31),15.9 (21),9.8 (10),4.6 (25),6.6 (9),21.1 (21),23.6 (3),7.8 (3)
2,Austin Spurs,113.3 (10),41.1 (11),94.4 (2),43.5 (26),13.0 (12),43.6 (7),29.8 (27),10.7 (13),14.2 (16),75.0 (16),15.8 (4),30.3 (27),46.1 (14),22.3 (23),14.0 (29),12.8 (1),5.6 (18),6.7 (7),16.6 (2),19.7 (20),7.2 (4)
3,Oklahoma City Blue,110.0 (17),37.0 (28),86.9 (24),42.6 (27),16.7 (3),45.1 (3),36.9 (4),12.1 (6),14.7 (11),82.6 (1),12.9 (13),34.2 (11),47.1 (10),24.7 (11),18.2 (7),8.9 (15),4.1 (28),7.3 (5),19.6 (12),19.7 (20),3.0 (11)
4,Sioux Falls Skyforce,116.8 (5),44.0 (3),87.7 (21),50.2 (1),12.0 (20),34.3 (25),35.0 (10),10.0 (18),12.9 (24),77.6 (9),11.0 (27),35.9 (4),46.9 (11),27.9 (2),17.2 (17),10.0 (6),5.3 (22),5.3 (22),19.7 (13),19.9 (19),10.6 (1)
5,Stockton Kings,115.1 (7),41.4 (9),88.4 (16),46.8 (9),15.6 (4),38.3 (11),40.8 (2),10.1 (17),13.5 (18),75.0 (16),13.5 (9),32.0 (19),45.5 (17),25.9 (7),16.3 (20),7.3 (29),6.1 (15),3.5 (31),20.3 (16),19.5 (24),3.9 (10)
6,Capital City Go-Go,111.0 (14),38.0 (26),80.9 (31),47.0 (8),11.1 (22),30.8 (28),36.0 (6),14.6 (1),17.9 (3),81.6 (3),11.7 (20),34.5 (9),46.2 (13),21.5 (28),22.8 (1),7.3 (29),7.4 (6),7.5 (4),22.2 (27),24.9 (1),2.6 (12)
7,Ciudad de Mexico Capitanes,106.0 (26),36.9 (30),83.4 (29),44.2 (19),13.2 (9),37.9 (12),34.9 (11),12.2 (4),16.1 (5),75.9 (13),9.8 (29),31.6 (20),41.3 (29),20.2 (29),20.2 (4),10.2 (5),5.4 (21),6.1 (16),19.3 (9),21.1 (9),-0.7 (16)
8,College Park Skyhawks,107.0 (23),39.1 (20),85.8 (27),45.6 (11),12.6 (15),37.7 (13),33.3 (15),10.0 (18),13.3 (21),75.0 (16),11.7 (20),36.0 (3),47.7 (6),21.8 (26),17.3 (15),7.4 (27),6.0 (16),4.8 (27),22.1 (26),20.0 (17),6.3 (6)
9,Valley Suns,124.4 (1),44.1 (2),92.0 (6),47.9 (2),18.7 (1),45.3 (2),41.2 (1),10.7 (13),14.4 (12),73.8 (21),8.9 (31),33.4 (13),42.3 (27),29.0 (1),13.4 (31),8.4 (21),4.6 (25),4.7 (28),22.6 (29),21.7 (5),6.4 (5)


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
27,Raptors 905,110.2 (15),42.3 (6),91.7 (8),46.2 (9),12.2 (18),36.2 (19),33.7 (13),8.1 (29),13.0 (23),62.4 (31),12.7 (15),29.0 (30),41.7 (28),26.3 (6),18.3 (6),12.4 (3),6.7 (10),5.7 (20),24.7 (31),21.4 (6),-9.0 (31)


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Greensboro Swarm']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
1,Greensboro Swarm,110.4 (14),39.6 (17),86.7 (24),45.6 (11),9.2 (30),27.1 (31),34.0 (12),14.2 (2),18.4 (2),77.1 (10),14.4 (7),35.2 (6),49.7 (3),18.0 (31),16.1 (21),9.2 (13),5.0 (24),6.4 (10),21.6 (22),23.7 (3),10.8 (1)


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Set up pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.expand_frame_repr', False)  # Prevent line breaks in the middle of the table

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Display the cleaned DataFrame
team_stats

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,Iowa Wolves,114.4,42.2,88.1,47.9,9.6,28.5,33.7,12.0,16.2,74.1,12.5,34.9,47.4,24.7,17.3,10.0,6.3,6.6,22.2,22.8,9.9
1,Greensboro Swarm,109.1,39.3,87.6,44.8,9.0,27.0,33.3,14.1,18.8,75.3,14.4,33.9,48.3,17.0,15.9,9.8,4.6,6.6,21.1,23.6,7.8
2,Austin Spurs,113.3,41.1,94.4,43.5,13.0,43.6,29.8,10.7,14.2,75.0,15.8,30.3,46.1,22.3,14.0,12.8,5.6,6.7,16.6,19.7,7.2
3,Oklahoma City Blue,110.0,37.0,86.9,42.6,16.7,45.1,36.9,12.1,14.7,82.6,12.9,34.2,47.1,24.7,18.2,8.9,4.1,7.3,19.6,19.7,3.0
4,Sioux Falls Skyforce,116.8,44.0,87.7,50.2,12.0,34.3,35.0,10.0,12.9,77.6,11.0,35.9,46.9,27.9,17.2,10.0,5.3,5.3,19.7,19.9,10.6
5,Stockton Kings,115.1,41.4,88.4,46.8,15.6,38.3,40.8,10.1,13.5,75.0,13.5,32.0,45.5,25.9,16.3,7.3,6.1,3.5,20.3,19.5,3.9
6,Capital City Go-Go,111.0,38.0,80.9,47.0,11.1,30.8,36.0,14.6,17.9,81.6,11.7,34.5,46.2,21.5,22.8,7.3,7.4,7.5,22.2,24.9,2.6
7,Ciudad de Mexico Capitanes,106.0,36.9,83.4,44.2,13.2,37.9,34.9,12.2,16.1,75.9,9.8,31.6,41.3,20.2,20.2,10.2,5.4,6.1,19.3,21.1,-0.7
8,College Park Skyhawks,107.0,39.1,85.8,45.6,12.6,37.7,33.3,10.0,13.3,75.0,11.7,36.0,47.7,21.8,17.3,7.4,6.0,4.8,22.1,20.0,6.3
9,Valley Suns,124.4,44.1,92.0,47.9,18.7,45.3,41.2,10.7,14.4,73.8,8.9,33.4,42.3,29.0,13.4,8.4,4.6,4.7,22.6,21.7,6.4


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Display the cleaned DataFrame
team_stats

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,Iowa Wolves,114.4,42.2,88.1,47.9,9.6,28.5,33.7,12.0,16.2,...,34.9,47.4,24.7,17.3,10.0,6.3,6.6,22.2,22.8,9.9
1,Greensboro Swarm,109.1,39.3,87.6,44.8,9.0,27.0,33.3,14.1,18.8,...,33.9,48.3,17.0,15.9,9.8,4.6,6.6,21.1,23.6,7.8
2,Austin Spurs,113.3,41.1,94.4,43.5,13.0,43.6,29.8,10.7,14.2,...,30.3,46.1,22.3,14.0,12.8,5.6,6.7,16.6,19.7,7.2
3,Oklahoma City Blue,110.0,37.0,86.9,42.6,16.7,45.1,36.9,12.1,14.7,...,34.2,47.1,24.7,18.2,8.9,4.1,7.3,19.6,19.7,3.0
4,Sioux Falls Skyforce,116.8,44.0,87.7,50.2,12.0,34.3,35.0,10.0,12.9,...,35.9,46.9,27.9,17.2,10.0,5.3,5.3,19.7,19.9,10.6
5,Stockton Kings,115.1,41.4,88.4,46.8,15.6,38.3,40.8,10.1,13.5,...,32.0,45.5,25.9,16.3,7.3,6.1,3.5,20.3,19.5,3.9
6,Capital City Go-Go,111.0,38.0,80.9,47.0,11.1,30.8,36.0,14.6,17.9,...,34.5,46.2,21.5,22.8,7.3,7.4,7.5,22.2,24.9,2.6
7,Ciudad de Mexico Capitanes,106.0,36.9,83.4,44.2,13.2,37.9,34.9,12.2,16.1,...,31.6,41.3,20.2,20.2,10.2,5.4,6.1,19.3,21.1,-0.7
8,College Park Skyhawks,107.0,39.1,85.8,45.6,12.6,37.7,33.3,10.0,13.3,...,36.0,47.7,21.8,17.3,7.4,6.0,4.8,22.1,20.0,6.3
9,Valley Suns,124.4,44.1,92.0,47.9,18.7,45.3,41.2,10.7,14.4,...,33.4,42.3,29.0,13.4,8.4,4.6,4.7,22.6,21.7,6.4


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Display the DataFrame
team_stats

,Unnamed: 0,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB RANK,REB RANK,AST RANK,TOV RANK,STL RANK,BLK RANK,BLKA RANK,PF RANK,PFD RANK,+/- RANK
0,1,Iowa Wolves,10,9,1,0.900,48.3,114.4,42.2,88.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Greensboro Swarm,8,7,1,0.875,48.0,109.1,39.3,87.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Austin Spurs,9,7,2,0.778,48.4,113.3,41.1,94.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Oklahoma City Blue,9,7,2,0.778,48.0,110.0,37.0,86.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,Sioux Falls Skyforce,9,7,2,0.778,48.4,116.8,44.0,87.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Stockton Kings,8,6,2,0.750,48.0,115.1,41.4,88.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Capital City Go-Go,10,7,3,0.700,48.0,111.0,38.0,80.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Ciudad de Mexico Capitanes,9,6,3,0.667,48.4,106.0,36.9,83.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,College Park Skyhawks,9,6,3,0.667,48.0,107.0,39.1,85.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,8,Valley Suns,9,6,3,0.667,48.4,124.4,44.1,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
team_stats.columns

Index(['Unnamed: 0', 'TEAM', 'GP', 'W', 'L', 'WIN%', 'MIN', 'PTS', 'FGM',
       'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB',
       'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', '+/-',
       'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
       'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
       'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
       'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
       'PF RANK', 'PFD RANK', '+/- RANK'],
      dtype='object')

In [4]:
team_stats = pd.read_html('https://www.basketball-reference.com/gleague/years/2025.html',
                 attrs={"id":"team-stats-sc-team"})[0]

ValueError: No tables found